In [ ]:
!pip install PyTelegramBotAPI

In [45]:
import telebot, torch, os, torchaudio, random
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from model import Model

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"{device} used")

cuda used


In [47]:
model = Model(n_output=10).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/models/voice_classification_weights.pth"))
model.eval()

Model(
  (conv1): Conv1d(1, 32, kernel_size=(80,), stride=(8,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=64, out_features=10, bias=True)
)

In [ ]:
!mkdir voice

In [44]:
model.load_state_dict(torch.load("/content/drive/MyDrive/models/voice_classification_weights.pth"))
model.eval()
bot = telebot.TeleBot("________YOUR TOKEN________")

names = ['Parisa', 'Zeinab', 'Alireza', 'Maryam', 'Morteza', 'Nahid', 'MohammadAli', 'Zahra', 'Sajjad', 'Hossein']
                     
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.send_message(message.chat.id, f"Hey {str(message.chat.first_name)} 🌻\nWelcome to the Audio Classification Bot🤍✌🏼")
    bot.send_message(message.chat.id, "Send me a voice\nI will try to tell you whose voice it is!")

    
@bot.message_handler(content_types=['voice'])
def voice(message):
    voice = bot.get_file(message.voice.file_id)
    downloaded_file = bot.download_file(voice.file_path)
    voice_path = voice.file_path

    with open(voice_path, 'wb') as audio_file:
        audio_file.write(downloaded_file)
    
    signal, sample_rate = torchaudio.load(voice_path)
    
    # preprocess
    signal = torch.mean(signal, dim=0, keepdim=True)
    transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=8000)
    signal = transform(signal)
    signal = signal.unsqueeze(0).to(device)
     
    # process
    preds = model(signal)
    
    # postprocess
    preds = preds.cpu().detach().numpy()
    output = np.argmax(preds)
    
    i = random.choice([1, 2, 3])
    if i == 1:
      bot.reply_to(message, "I think you are " + names[output] + " 🧐")
    elif i == 2:
      bot.reply_to(message, "Your voice is like the voice of " + names[output] + " 🤔")
    else:
      bot.reply_to(message, "I guess you are " + names[output] + " 👀")

bot.polling()